In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
import tensorflow as tf
from transformers import TFDistilBertForSequenceClassification, DistilBertTokenizerFast
from functools import partial
from tqdm.notebook import tqdm
from IPython.display import HTML

from transformers_gradients.plotting import visualise_explanations_as_html
from transformers_gradients.text_classification.explanation_func import (
    gradient_norm,
    gradient_x_input,
    integrated_gradients,
    smooth_grad,
    noise_grad,
    noise_grad_plus_plus,
)
from transformers_gradients.config import (
    SmoothGradConfing,
    NoiseGradConfig,
    NoiseGradPlusPlusConfig,
)

tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
model = TFDistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english"
)

tokenizer = DistilBertTokenizerFast.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english"
)

Metal device set to: Apple M1 Pro


All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [4]:
x = ["Like four times a year I rediscover Björk and listen to her full discography"]
y = [1]

In [5]:
a_batch = [
    f(model, x, y, tokenizer)[0]
    for f in tqdm(
        [
            gradient_norm,
            gradient_x_input,
            integrated_gradients,
            partial(smooth_grad, config=SmoothGradConfing(explain_fn="GradNorm")),
            partial(noise_grad, config=NoiseGradConfig(explain_fn="GradNorm")),
            partial(
                noise_grad_plus_plus,
                config=NoiseGradPlusPlusConfig(explain_fn="GradNorm"),
            ),
            partial(smooth_grad, config=SmoothGradConfing(explain_fn="GradXInput")),
            partial(noise_grad, config=NoiseGradConfig(explain_fn="GradXInput")),
            partial(
                noise_grad_plus_plus,
                config=NoiseGradPlusPlusConfig(explain_fn="GradXInput"),
            ),
            smooth_grad,
            noise_grad,
            noise_grad_plus_plus,
        ]
    )
]

  0%|          | 0/12 [00:00<?, ?it/s]

2023-04-13 09:45:04.688096: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [8]:
HTML(
    visualise_explanations_as_html(
        a_batch,
        labels=[
            "Gradient Norm",
            "Gradient X Input",
            "Integrated Gradients",
            "Smooth Grad + Gradient Norm",
            "Noise Grad + Gradient Norm",
            "NoiseGrad++ + Gradient Norm",
            "Smooth Grad + Gradient X Input",
            "Noise Grad + Gradient X Input",
            "NoiseGrad++ + Gradient X Input",
            "Smooth Grad + Integrated Gradients",
            "Noise Grad + Integrated Gradients",
            "NoiseGrad++ + Integrated Gradients",
        ],
    )
)